[View in Colaboratory](https://colab.research.google.com/github/yepuv1/StockPerformanceClassification/blob/master/Prime_numbers.ipynb)

In [0]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops

%matplotlib inline
np.random.seed(1)

In [2]:
print(tf.__version__)

1.11.0-rc2
